# XGBoost Endpoint - Multiple Models hosted on same instance
<h4>Invoke Specific Model</h4>

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
import json

import boto3
import re
from sagemaker import get_execution_role
import sagemaker

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [2]:
# We need to specify the location of each model - relative path is fine
# TODO - Update your path here
targetModels = ["xgboost-bikerental-hyper-one-2021-06-29-19-03-51-772/output/model.tar.gz",
                "xgboost-bikerental-hyper-two-2021-06-29-19-08-04-688/output/model.tar.gz"]

In [3]:
# Create a predictor and point to an existing endpoint (note this endpoint has two versions of the model)
endpoint_name = 'xgboost-bikerental-hyper'
predictor = sagemaker.predictor.Predictor (endpoint_name=endpoint_name)
predictor.serializer = CSVSerializer()

In [4]:
# Read Test Data
df_all = pd.read_csv('bike_test.csv')

In [5]:
df_all.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4


In [6]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1],[20,1]]
arr_test = df_all[df_all.columns[1:]].values

In [7]:
arr_test.shape

(6493, 13)

### Invoke Model One

In [9]:
# target_model = path to the model artifact. For multi-model endpoints, we need to provide the path to the model artifact
# Call the first model
# Output is a JSON List
result = predictor.predict(
    arr_test[:5], 
    target_model=targetModels[0])

print(result)

b'[2.332122325897217, 1.9005593061447144, 1.6316101551055908, 1.0376962423324585, 0.9574321508407593]'


### Invoke Model Two

In [11]:
# target_model = path to the model artifact. For multi-model endpoints, we need to provide the path to the model artifact
# Call the second model
# Output is a JSON List
result = predictor.predict(
    arr_test[:5], 
    target_model=targetModels[1])

print(result)

b'[2.489321708679199, 1.7995195388793945, 1.92416250705719, 1.335949182510376, 1.1886606216430664]'


### Split the input data into chunks
There are thousands of rows in this data set for which need inference.  
When communicating over internet, it is a good idea to split the data into chunks to prevent payload and timeout error

In [12]:
def inference_by_version(targetModel = None):
    # For large number of predictions, we can split the input data and
    # Query the prediction service.
    # array_split is convenient to specify how many splits are needed
    predictions = []
    for arr in np.array_split(arr_test,10):
        result = predictor.predict(arr, target_model=targetModel)
        result = json.loads(result.decode("utf-8"))
        print (arr.shape)
        predictions += [float(r) for r in result]
        
    return predictions

### Use all available variants

In [13]:
print('model one inference')
df_all["count_hyper_one"] = np.expm1(inference_by_version(targetModel=targetModels[0]))
print('model two inference')
df_all["count_hyper_two"] = np.expm1(inference_by_version(targetModel=targetModels[1]))

model one inference
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
model two inference
(650, 13)
(650, 13)
(650, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)
(649, 13)


In [14]:
df_all[["count_hyper_one","count_hyper_two"]].describe()

,count_hyper_one,count_hyper_two
count,6493.000000,6493.000000
mean,183.412390,184.068854
std,171.869936,175.669853
min,0.419208,0.718337
25%,41.106633,39.230815
50%,141.825441,138.852258
75%,273.490148,272.473081
max,942.930326,902.084974


In [15]:
# Delete Endpoint to prevent unnecessary charges
predictor.delete_endpoint()